In [9]:
from pymongo import MongoClient
import requests
import datetime

client = MongoClient('ec2-52-211-76-129.eu-west-1.compute.amazonaws.com',27017)
db = client.wax_dfg
deputes_db = db.deputes

In [18]:
deputes_enmandat_from_api = requests.get('https://www.nosdeputes.fr/deputes/enmandat/json')
deputes_enmandat_json = deputes_enmandat_from_api.json()
deputes_db.drop()

In [19]:
for depute in deputes_enmandat_json['deputes']:   
        url_depute_json = depute['depute']['url_nosdeputes_api']
        depute_from_api = requests.get(url_depute_json)
        depute_json = depute_from_api.json()
        departement = db.departement.find({'DEP': depute_json['depute']['num_deptmt']})
        if(departement.count()==0):
            depute_json['depute']['region'] ={'num': depute_json['depute']['num_deptmt'], 'nom':'Français établis hors de France'}
        else:
            depute_json['depute']['region'] ={'num': departement[0]['REGION']['REGION'], 'nom':departement[0]['REGION']['NCCENR']}
        depute_json['depute']['date_naissance'] = datetime.datetime.strptime(depute_json['depute']['date_naissance'], '%Y-%m-%d')
        deputes_db.insert_one(depute_json['depute'])

http://www.nosdeputes.fr/jean-francois-lamour/json
http://www.nosdeputes.fr/jean-michel-clement/json
http://www.nosdeputes.fr/bernard-reynes/json
http://www.nosdeputes.fr/marc-laffineur/json
http://www.nosdeputes.fr/charles-de-courson/json
http://www.nosdeputes.fr/christian-paul/json
http://www.nosdeputes.fr/jean-pierre-dufau/json
http://www.nosdeputes.fr/herve-feron/json
http://www.nosdeputes.fr/jean-louis-touraine/json
http://www.nosdeputes.fr/paul-giacobbi/json
http://www.nosdeputes.fr/marc-le-fur/json
http://www.nosdeputes.fr/jean-rene-marsac/json
http://www.nosdeputes.fr/jean-leonetti/json
http://www.nosdeputes.fr/valerie-fourneyron/json
http://www.nosdeputes.fr/francois-de-rugy/json
http://www.nosdeputes.fr/jerome-chartier/json
http://www.nosdeputes.fr/renaud-gauquelin/json
http://www.nosdeputes.fr/francis-hillmeyer/json
http://www.nosdeputes.fr/patrick-lemasle/json
http://www.nosdeputes.fr/jerome-lambert/json
http://www.nosdeputes.fr/patrice-verchere/json
http://www.nosdeputes.f